# Machine Learning: Anime Recommendation

#### Data Collection

In [1]:
import kaggle

In [2]:
!kaggle datasets download -d CooperUnion/anime-recommendations-database --unzip


  0%|          | 0.00/25.0M [00:00<?, ?B/s]
  4%|3         | 1.00M/25.0M [00:00<00:02, 8.98MB/s]
  8%|7         | 2.00M/25.0M [00:00<00:02, 9.33MB/s]
 12%|#1        | 3.00M/25.0M [00:00<00:02, 9.64MB/s]
 20%|#9        | 5.00M/25.0M [00:00<00:02, 10.3MB/s]
 24%|##3       | 6.00M/25.0M [00:00<00:02, 9.88MB/s]
 28%|##7       | 7.00M/25.0M [00:00<00:02, 9.38MB/s]
 36%|###5      | 9.00M/25.0M [00:01<00:02, 8.02MB/s]
 44%|####3     | 11.0M/25.0M [00:01<00:01, 8.56MB/s]
 52%|#####1    | 13.0M/25.0M [00:01<00:01, 9.19MB/s]
 56%|#####5    | 14.0M/25.0M [00:01<00:01, 9.45MB/s]
 60%|#####9    | 15.0M/25.0M [00:01<00:01, 9.61MB/s]
 68%|######7   | 17.0M/25.0M [00:01<00:00, 10.2MB/s]
 72%|#######1  | 18.0M/25.0M [00:01<00:00, 10.2MB/s]
 76%|#######5  | 19.0M/25.0M [00:02<00:00, 9.95MB/s]
 80%|#######9  | 20.0M/25.0M [00:02<00:00, 9.64MB/s]
 88%|########7 | 22.0M/25.0M [00:02<00:00, 10.5MB/s]
 96%|#########5| 24.0M/25.0M [00:02<00:00, 10.9MB/s]
100%|##########| 25.0M/25.0M [00:02<00:00, 10.0MB/s]


##### Libraries

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set()
plt.style.use('seaborn-whitegrid')
sns.set_style("white")

import scipy.stats as st